Keras Tuner는 TensorFlow 프로그램에 대한 최적의 하이퍼파라미터 세트를 선택하는 데 도움을 주는 라이브러리

머신러닝(ML) 애플리케이션에 대한 올바른 하이퍼파라미터 세트를 선택하는 과정을 하이퍼파라미터 조정 또는 하이퍼튜닝이라 일컫음

### Keras Tuner를 사용하여 이미지 분류 애플리케이션에 하이퍼튜닝을 수행

In [37]:
import tensorflow as tf
from tensorflow import keras

import IPython

In [38]:
!pip install keras-tuner --upgrade

Requirement already up-to-date: keras-tuner in c:\users\chaeh\appdata\local\programs\python\python39-32\lib\site-packages (1.0.3)


You should consider upgrading via the 'c:\users\chaeh\appdata\local\programs\python\python39-32\python.exe -m pip install --upgrade pip' command.


In [39]:
import kerastuner as kt

ModuleNotFoundError: No module named 'kerastuner'

- 데이터세트 다운로드 및 준비

In [27]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

In [28]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

- 모델 정의하기

In [29]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value = 32, max_value = 512, step = 32)
  model.add(keras.layers.Dense(units = hp_units, activation = 'relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer 
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values = [1e-2, 1e-3, 1e-4]) 

  model.compile(optimizer = keras.optimizers.Adam(learning_rate = hp_learning_rate),
                loss = keras.losses.SparseCategoricalCrossentropy(from_logits = True), 
                metrics = ['accuracy'])

  return model

- 튜너를 인스턴스화하고 하이퍼튜닝 수행하기

In [30]:
tuner = kt.Hyperband(model_builder,
                     objective = 'val_accuracy', 
                     max_epochs = 10,
                     factor = 3,
                     directory = 'my_dir',
                     project_name = 'intro_to_kt')

NameError: name 'kt' is not defined

In [31]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)

In [32]:
tuner.search(img_train, label_train, epochs = 10, validation_data = (img_test, label_test), callbacks = [ClearTrainingOutput()])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials = 1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

NameError: name 'tuner' is not defined

In [33]:
# Build the model with the optimal hyperparameters and train it on the data
model = tuner.hypermodel.build(best_hps)
model.fit(img_train, label_train, epochs = 10, validation_data = (img_test, label_test))

NameError: name 'tuner' is not defined